In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [34]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input


In [35]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   preprocessing_function=preprocess_input)
                                  # validation_split= 0.2)

training_set = train_datagen.flow_from_directory("mel_sprec\mel_11\\train",
                                            batch_size=15,
                                            shuffle= True,
                                            target_size=(64,64))

val_datagen = ImageDataGenerator(rescale = 1./255,
                                preprocessing_function=preprocess_input)

validation_set = val_datagen.flow_from_directory("mel_sprec\mel_11\\validation",
                                            batch_size=15,
                                            shuffle = True,
                                            target_size=(64,64))

Found 300 images belonging to 6 classes.
Found 102 images belonging to 6 classes.


In [36]:
feature_extractor = EfficientNetB0(weights='imagenet', 
                             input_shape=(64,64, 3),
                             include_top=False)

# Set this parameter to make sure it's not being trained
feature_extractor.trainable = True

# Set the input layer
input_ = tf.keras.Input(shape=(64,64, 3))

# Set the feature extractor layer
x = feature_extractor(input_, training= True)

# Set the pooling layer
x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = tf.keras.layers.Flatten()(x)

#x = tf.keras.layers.Dropout(0.2)(x)

#x = tf.keras.layers.Dense(256,activation="relu")(x)

x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Dense(64,activation="relu")(x)



# Set the final layer with sigmoid activation function
output_ = tf.keras.layers.Dense(6, activation='softmax')(x)

# Create the new model object
model = tf.keras.Model(input_, output_)

# Compile it
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00025 , decay = 1e-6),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# Print The Summary of The Model
model.summary()

Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 64, 64, 3)]       0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 2, 2, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d_8 ( (None, 1280)              0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 64)                81984     
_________________________________________________________________
dense_20 (Dense)             (None, 6)               

In [12]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [17]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch/20)
)

In [37]:
history = model.fit(training_set, epochs=20, validation_data=validation_set)

Epoch 1/20
20/20 [==============================] - 6s 319ms/step - loss: 1.7792 - accuracy: 0.2633 - val_loss: 1.5753 - val_accuracy: 0.3922
Epoch 2/20
20/20 [==============================] - 5s 244ms/step - loss: 1.4430 - accuracy: 0.4600 - val_loss: 1.2956 - val_accuracy: 0.5000
Epoch 3/20
20/20 [==============================] - 5s 245ms/step - loss: 1.2018 - accuracy: 0.5800 - val_loss: 1.2598 - val_accuracy: 0.4902
Epoch 4/20
20/20 [==============================] - 5s 243ms/step - loss: 1.0231 - accuracy: 0.6600 - val_loss: 1.2222 - val_accuracy: 0.5098
Epoch 5/20
20/20 [==============================] - 5s 251ms/step - loss: 0.8328 - accuracy: 0.7200 - val_loss: 1.0941 - val_accuracy: 0.6176
Epoch 6/20
20/20 [==============================] - 5s 243ms/step - loss: 0.6793 - accuracy: 0.7567 - val_loss: 0.9780 - val_accuracy: 0.6961
Epoch 7/20
20/20 [==============================] - 5s 245ms/step - loss: 0.6280 - accuracy: 0.7733 - val_loss: 1.0679 - val_accuracy: 0.5882
Epoch 

{'loss': [0.8539015650749207,
  0.76399827003479,
  0.6751458048820496,
  0.6121060252189636,
  0.5596533417701721,
  0.5018951296806335,
  0.4952899217605591,
  0.4094383716583252,
  0.4490027129650116,
  0.3979167938232422,
  0.3232474625110626,
  0.3202124536037445,
  0.2565664052963257,
  0.2541413903236389,
  0.24831071496009827,
  0.20706701278686523,
  0.19852091372013092,
  0.11419669538736343,
  0.10718680918216705,
  0.10221825540065765],
 'accuracy': [0.7149099111557007,
  0.7416712045669556,
  0.7602403163909912,
  0.7788093686103821,
  0.7979246377944946,
  0.8099399209022522,
  0.81922447681427,
  0.8547241687774658,
  0.8323320746421814,
  0.8569087982177734,
  0.8973238468170166,
  0.8842162489891052,
  0.9098852872848511,
  0.9208083152770996,
  0.9142544865608215,
  0.9240851998329163,
  0.9377389550209045,
  0.9617695212364197,
  0.9606772065162659,
  0.9606772065162659],
 'val_loss': [0.7867847681045532,
  0.7295507788658142,
  0.5586274862289429,
  0.67690074443817

In [19]:
model.save("Unbalanced.h5")

In [23]:
history1 = history

In [25]:
history2= history